# Workflow for building temperature Profiles

In [1]:
##Import Libraries##
from __future__ import division, unicode_literals, print_function  # for compatibility with Python 2 and 3
from pandas import DataFrame, Series  # for convenience
from PIL import Image
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pims
import trackpy as tp
import cv2
import os
from scipy.optimize import curve_fit
import matplotlib.patches as mpatches
from scipy.stats import norm
import statistics
from matplotlib.widgets import Slider
import random

In [5]:
default_rc = dict(mpl.rcParams) #safe the default plotting settings into a variable

In [6]:
##Optionally, tweak styles.##
mpl.rc('figure',  figsize=(10, 10*(1236/1376))) #400/375
mpl.rc('image', cmap='gray')

In [8]:
##______________________________________________________________________________________________________________________________________________##

###Convert Video to Image Sequence and save to directory###

#File path of video that needs converted
folderPath = r'g:\My Drive\workAppState\prj_shocks\expData\data20230419temperatureProfileTest\inputs'
#Filepath for directory to save image sequence
videoName = r'2mA_420V_12sccm_111mtorr_135volt_bias.avi'
#Video Type
vidType = '.avi'
#Image Type
imgType = '.png'

##CODE DOES NOT NEED ALTERED FOR VIDEO/IMAGE-SEQUENCE CONVERTER PAST THIS POINT
#Read the video from specified path
    
cam = cv2.VideoCapture(folderPath + r'\Video' + videoName + vidType)
  
try:
      
    # creating a folder named 
    direcPath = folderPath + r'\ImageSequences' + videoName
    if not os.path.exists(direcPath):
        os.makedirs(direcPath)
  
# if not created then raise error
except OSError:
    print ('Error: Creating directory of data')
  
# frame
currentframe = 0
  
while(True):
      
    # reading from frame
    ret,frame = cam.read()
  
    if ret:
        # if video is still left continue creating images
        name = direcPath + r'\frame' + str(currentframe) + imgType
        print ('Creating...' + name)
  
        # writing the extracted images
        cv2.imwrite(name, frame)
  
        # increasing counter so that it will
        # show how many frames are created
        currentframe += 1
    else:
        break
  
# Release all space and windows once done
cam.release()
cv2.destroyAllWindows()


In [9]:
##______________________________________________________________________________________________________________________________________________##

###Create Directory for Plots###

if not os.path.exists(folderPath + r'\Plots' + videoName):
    os.makedirs(folderPath + r'\Plots' + videoName)
    


In [10]:
##______________________________________________________________________________________________________________________________________________##

###Import image sequence and convert to grayscale###

#Define function that converts to grayscale
@pims.pipeline
def gray(image):
    return image[:, :, 1]  # Take just the green channel


#Creates an array for grayscale frames
frames = []

#Access first image in directory as test image
testim = gray(cv2.imread(direcPath + r'\frame' + str(0) + imgType))
print(" ")
print("View of current image before cropping provided in plots tab")
print(" ")
plt.imshow(testim)
plt.show()
print("Recommended starting scale is x=525:900 y=375:775")
print("(Press enter when prompted for input to use recommended scale)")
#Loop to test cropping scale and then apply to entire directory
while True:
    #Input for cropping scale
    xFrames1 = int(input("Input min x pixel coordinate: ") or "550")
    xFrames2 = int(input("Input max x pixel coordinate: ") or "925")
    yFrames1 = int(input("Input min y pixel coordinate: ") or "450")
    #yFrames2= int(input("Input max y pixel coordinate: ") or "775")
    yFrames2 = int(((xFrames2-xFrames1)*(1236/1376))+yFrames1)
    #Crops test images
    exFrame = testim[yFrames1:yFrames2, xFrames1:xFrames2]
    #Shows cropped image
    plt.imshow(exFrame)
    plt.show()
    #Request to proceed
    answer = input("Would you like to reset scale? (yes or no): ")
    if answer=="yes":
        #Restarts loop for new crop scale
        continue 
    else:
        #Loop to apply cropping to all images in directory
        for i in range(currentframe):
            im = gray(cv2.imread(direcPath + r'\frame' + str(i) + imgType))
            frames.append(im[yFrames1:yFrames2, xFrames1:xFrames2])
        break

plt.imshow(frames[0])


TypeError: 'NoneType' object is not subscriptable